In [3]:
from model.UNet import UNet
from model.DeepLabV3Plus import DeepLabV3Plus
from model.HRNet import HighResolutionNet

import os

from PIL import Image

import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [5]:
class SegmentationDataset(Dataset):
  def __init__(self, data_type, mode, transform=None):    
    self.mode = mode
    self.transform = transform
    self.input, self.label = [], []

    # 작업할 폴더의 경로
    path = './dataset/{0}s'.format(data_type) 

    self.input_path = os.path.join(path, self.mode, "data")
    self.label_path = os.path.join(path, self.mode, "label")

    # 폴더 안에 있는 데이터 파일들의 이름을 추출
    data_names = [name.split('.')[0] for name in os.listdir(self.input_path)]

    # 데이터 전처리
    for data_name in data_names:
        self.input.append('{0}/{1}.png'.format(self.input_path, data_name))
        self.label.append('{0}/{1}.png'.format(self.label_path, data_name))

  def __len__(self):
    return len(self.input)

  def __getitem__(self, index):
    input = Image.open(self.input[index])
    label = Image.open(self.label[index])
    
    if self.transform:
      input = self.transform(input)
      label = self.transform(label)

    return input, label

In [6]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset   = SegmentationDataset('building', 'train', transform)
test_dataset    = SegmentationDataset('building', 'test',  transform)

In [7]:
# 저장된 모델 가중치 로드
#model.load_state_dict(torch.load(os.path.join(config["output_dir"], config["model_name"])) 

In [8]:
output_dir_path = "./trained"

def train(model, dataset, num_epoch = 10):
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True) 

    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters()) 

    for epoch in range(num_epoch):
        model.train()

        costs = []
        
        for batch in dataloader:
            input, label = batch
            
            input = input.to(device)
            label = label.to(device)

            hypothesis = model(input)
 
            cost = loss_func(hypothesis, label)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            costs.append(cost.item())

        #print("{0}/{1} : {2}".format(epoch, num_epoch, np.mean(costs)))
        
        #torch.save(model.state_dict(), os.path.join(output_dir_path, "{0}_epoch_{1:d}.pt".format(type(model).__class__.__name__, epoch + 1)))


In [10]:
num_epoch = 10

#model1 = UNet().to(device)
#model2 = DeepLabV3Plus().to(device)
model3 = HighResolutionNet().to(device)

#train(model1, train_dataset, num_epoch)
#train(model2, train_dataset, num_epoch)
train(model3, train_dataset, num_epoch)

RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target' in call to _thnn_nll_loss_forward

In [21]:
from tqdm import tqdm
import gc
# Clear unused CUDA memory

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

torch.cuda.empty_cache()
gc.collect()

# Initialize HRNetV2 model
model3 = HighResolutionNet().to(device=device)

transform = transforms.Compose([transforms.ToTensor()])

train_dataset   = SegmentationDataset('building', 'train', transform)
test_dataset    = SegmentationDataset('building', 'test',  transform)

BATCH_SIZE = 8
LEARNING_RATE = 0.001
NUM_EPOCHS = 10

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model3.parameters(), lr=LEARNING_RATE)

# Clear unused CUDA memory
torch.cuda.empty_cache()
for epoch in range(NUM_EPOCHS):
    model3.train()  # Set the model to train mode
    
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward pass
        outputs = model3(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        running_loss += loss.item() * inputs.size(0)
    
    # Calculate average loss for epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {epoch_loss:.4f}")
    
    torch.cuda.empty_cache()
    gc.collect()


  0%|          | 0/155 [00:16<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 8.00 GiB total capacity; 21.62 GiB already allocated; 0 bytes free; 21.74 GiB reserved in total by PyTorch)